# La Grande Fuga
Sulla base del libro di Angus Deaton "La Grande Fuga - salute, ricchezza e origini della disuguaglianza", sarà replicata a livello base l'analisi svolta approfondendo le dinamiche di "speranza di vita" e "reddito" che caratterizzano l'Italia e le regioni italiane

- Open Data -> ISTAT

In [1]:
# Import librerie per analisi dati (Pandas) e dati Istat
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import istat
import jsonstat

# cache dir per velocizzare analisi in locale
cache_dir = os.path.abspath(os.path.join("..", "tmp/od_la_grande_fuga", "istat_cached"))
istat.cache_dir(cache_dir)
istat.lang(0)  # lingua italiano
print("cache_dir is '{}'".format(istat.cache_dir()))

cache_dir is '/home/acesarini/opendata/tmp/od_la_grande_fuga/istat_cached'


In [2]:
# Lista Aree
# istat.areas()
istat_area_pop = istat.area(8)

## Speranza di vita alla nascita

In [4]:
# Lista Dataset - Popolazione -> KPI: Speranza di vita alla nascita
istat_dataset_popolazione = istat_area_pop.dataset('DCIS_INDDEMOG1')
istat_dataset_popolazione

nr,name,nr. values,values (first 3 values)
0,Territorio,140,"1:'Italia', 3:'Nord', 4:'Nord-ovest' ..."
1,Tipo indicatore,22,"64:'tasso di natalità (per mille abitanti)', 65:'tasso di mortalità (per mille abitanti)', 66:'tasso di nuzialità (per mille abitanti)' ..."
2,Anno,15,"2148:'2013', 2053:'2008', 1958:'2003' ..."


In [5]:
# Lista di tutte le Dimensioni
#istat_dataset_popolazione.dimensions()

In [6]:
# Speranza di Vita Maschi
spec1 = {
    #"Territorio":1,                 # 1 -> Italia
    #"Anno":2167,                   # ALL
    "Tipo indicatore":75            # 75 -> Speranza di vita alla nascita - maschi
}

collection1 = istat_dataset_popolazione.getvalues(spec1)
ds = collection1.dataset(0)
df_m = ds.to_data_frame('Territorio')

In [7]:
# Speranza di Vita Femmine
spec2 = {
    #"Territorio":1,                # 1 -> Italia
    #"Anno":2167,                   # ALL
    "Tipo indicatore":77            # 77 -> Speranza di vita alla nascita - femmine
}

collection2 = istat_dataset_popolazione.getvalues(spec2)
ds2 = collection2.dataset(0)
df_f = ds2.to_data_frame('Territorio')

In [8]:
df_f.count()

Anno     1946
Value    1946
dtype: int64

In [9]:
df_m.count()

Anno     1946
Value    1946
dtype: int64

In [50]:
# Calcolo Speranza di Vita alla Nascita (media tra maschi e femmine) - Territorio Italia, Nord e Sud
df_merge = pd.concat([df_m, df_f], axis=1)
df_merge.columns = ['Anno', 'Value_M', 'Anno_F', 'Value_F']
df_merge = df_merge.drop('Anno_F', 1)
df_merge.reset_index(level=0, inplace=True)

def value(x):
    return round(((x['Value_F'] + x['Value_M'])/2),1)

df_merge['Value'] = df_merge.apply(lambda x: value(x), axis=1)
df_merge = df_merge.drop('Value_F', 1)
df_merge = df_merge.drop('Value_M', 1)

df_merge=df_merge[(df_merge['Territorio']=='Italia') |
                  (df_merge['Territorio']=='Nord') |
                  (df_merge['Territorio']=='Sud')]

In [49]:
# df_merge.head(2)

# writer = pd.ExcelWriter('Vita.xlsx')
# df_merge.to_excel(writer,'vita')
# writer.save()


,Territorio,Anno,Value
0,Italia,2002,80.1
1,Italia,2003,80.0
2,Italia,2004,80.8
3,Italia,2005,80.8
4,Italia,2006,81.2
5,Italia,2007,81.2
6,Italia,2008,81.3
7,Italia,2009,81.5
8,Italia,2010,81.8
9,Italia,2011,82.0


In [52]:
# Directory dove salvare i file, da utilizzare in data_visualization
dir_df = os.path.join(os.path.abspath(''),'stg')

df_merge_filename = r'df_vita.pkl'
df_merge_fullpath = os.path.join(dir_df, df_merge_filename)
df_merge.to_pickle(df_merge_fullpath)

## Reddito pro capite

In [56]:
istat_area_cn = istat.area(17)
istat_dataset_reddito = istat_area_cn.dataset('DCCN_TNA')
istat_dataset_reddito

nr,name,nr. values,values (first 3 values)
0,Territorio,142,"1:'Italia', 2:'Centro-nord', 3:'Nord' ..."
1,Tipo aggregato,22,"2:'spesa per consumi finali sul territorio economico delle famiglie residenti e non residenti', 5:'spesa per consumi finali delle amministrazioni pubbliche', 8:'investimenti fissi lordi, interni' ..."
2,Branca di attività (NACE Rev2),43,"128:'fabbricazione di coke e prodotti derivanti dalla raffinazione del petrolio, fabbricazione di prodotti chimici e farmaceutici', 192:'attività immobiliari', 130:'industrie tessili, confezione di articoli di abbigliamento e di articoli in pelle e simili' ..."
3,Attività non finanziarie,3,"3:'totale capitale fisso per tipo di attività', 4:'totale costruzioni', 79:'non applicabile' ..."
4,Funzione di spesa (coicop/cofog),28,"64:'istruzione', 2:'alimentari e bevande non alcoliche', 3:'bevande alcoliche, tabacco,narcotici' ..."
5,Valutazione,3,"16:'valori concatenati con anno di riferimento 2010', 9:'prezzi correnti', 10:'prezzi dell'anno precedente' ..."
6,Correzione,1,1:'dati grezzi'
7,Tipologia di prezzo,2,"1:'prezzi base', 7:'non applicabile'"
8,Edizione,3,"2203:'Nov-2015', 2223:'Dic-2016', 2183:'Ott-2014' ..."
9,Tempo e frequenza,21,"1920:'2001', 1825:'1996', 2148:'2013' ..."


In [76]:
#istat_dataset_reddito.dimensions()

In [112]:
# Prodotto Interno Lordo ai prezzi di mercato per abitante

spec2 = {
#"Territorio":1,
"Tipo aggregato":27,
#"Branca di attività (NACE Rev2)":128,
#"Attività non finanziarie":3,
#"Funzione di spesa (coicop/cofog)":3,
"Valutazione":9,
#"Correzione":1,
#"Tipologia di prezzo":1,
"Edizione":2223
#"Tempo e frequenza":1920
}

collection2 = istat_dataset_reddito.getvalues(spec2)
#ds = collection.dataset(0)

In [113]:
ds = collection2.dataset(0)
df_red = ds.to_data_frame('Tempo e frequenza')
df_red.reset_index(level=0, inplace=True)
df_red = df_red[['Tempo e frequenza','Territorio', 'Value']]

In [114]:
df_red=df_red[(df_red['Territorio']=='Italia') |
                  (df_red['Territorio']=='Nord') |
                  (df_red['Territorio']=='Sud')]

In [115]:
#writer = pd.ExcelWriter('Reddito.xlsx')
#df_red.to_excel(writer,'reddito')
#writer.save()

In [116]:
df_red_filename = r'df_reddito.pkl'
df_red_fullpath = os.path.join(dir_df, df_red_filename)
df_red.to_pickle(df_red_fullpath)